# **Simulation of Smart Grid**

Install and import mosaik    -- ( **DON'T USE. BROKE STUFF** )

In [1]:
!pip install mosaik

import mosaik


#DON'T USE MOSAIK ANYMORE. INTEGRATION WITH IT BROKE STUFF

     |████████████████████████████████| 71kB 4.9MB/s 


Install krangpower (To simulate grid based on OpenDDS)    (https://pypi.org/project/krangpower/)

In [2]:
!pip install krangpower

# https://pypi.org/project/krangpower/0
# Docs : https://krangpower.readthedocs.io/en/master/usage.html

     |████████████████████████████████| 215kB 7.6MB/s 
     |████████████████████████████████| 92kB 6.1MB/s 
     |████████████████████████████████| 204kB 15.1MB/s 
     |████████████████████████████████| 133kB 20.4MB/s 
     |████████████████████████████████| 7.8MB 11.4MB/s 
  Created wheel for OpenDSSDirect.py: filename=OpenDSSDirect.py-0.4.0-cp36-none-any.whl size=65712 sha256=a830108eca82e7429c6fde7c663203e2900cfa41d735a62991eecb62a60ebcd7
  Stored in directory: /root/.cache/pip/wheels/15/be/cf/69ac9f38f37f7d6034bcaf471ea851473fa46962a19615ef59
  Created wheel for frozendict: filename=frozendict-1.2-cp36-none-any.whl size=3149 sha256=93a627f7fab19efad76ed7825135e63fb65a3c5708b9e39fb71c2a9f2ab27bb4
  Stored in directory: /root/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
Successfully built OpenDSSDirect.py frozendict


Import essential libraries and define units to use the pint library

In [3]:
import numpy as np
import numpy.fft as fft
from PIL import Image
import matplotlib.pyplot as plt
import cvxpy as cp  # For optimization

import krangpower as kp  # This is our main library
unit = kp.UM    # Pint library for units
# https://pypi.org/project/Pint/
# Docs : https://pint.readthedocs.io/en/stable/

# Start with our grid

In [4]:
src = kp.Vsource(basekv=220*unit.V, frequency=100.0) # Make a voltage source with voltage 220 V and frquency 100 Hz

In [5]:
grid = kp.Krang('theGrid', voltage_source=src)  # Instantiate a grid as a krang object (we can have only one of those) with our source above

In [6]:
mainLine = kp.LineCode_S('mainTransmission', nphases=3) # Declare a transmission line

In [7]:
grid << mainLine
grid['sourcebus', 'alpha']

<BusView['sourcebus', 'alpha']>

## The prosumer class

In [8]:
class prosumer(kp.DecisionModel):

  def __init__(self, id, production, consumption, solarIrradiance):  # All the attributes would take their values from dataset - both initially and on each iteration
    self.id = id
    self.production = production
    self.consumption = consumption
    self.solarIrradiance = solarIrradiance

    prosumerLine = kp.LineCode_S(str(id)+'Transmission', nphases=3).aka(str(id)+'Transmisssion') # Make a transmission line in the internal prosumer model
    grid['mainTransmission', 'alpha'] <<kp.PvSystem(phases =3, irradiance = solarIrradiance).aka('prosumerPV'+str(id))
    # Add a solar photovoltaic that uses the irradiance from the dataset to generate power


  
  def update(self, oek, mynode):
    hour = oek.brain.Solution.DblHour()  # Extracts which hour the simulation is currently on

    # Update cpmsumption and irradiance from dataset

    # Optimization by scheduling algo goes here

    # Update production as per scheduling algo
  
  





## Making a grid with prosumers

In [11]:
transmission1 = kp.LineCode_S('transmission1', nphases=3)
grid<<transmission1

#prosumer1 = prosumer(id = 1, production =10, consumption=5, solarIrradiance =30.0*unit.kilowatt/(unit.metre)**2)

prosumerList = []
#prosumerList.append(prosumer1)

for i in range (2, 101):
  prosumerList.append(prosumer(id = i,  production =10, consumption=5, solarIrradiance =30.0*unit.kilowatt/(unit.metre)**2))